In [17]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import datetime as dt
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('../UF 1990-2025.csv')
df

,date,clfclp
0,1990-01-01,5435.28
1,1990-01-02,5438.23
2,1990-01-03,5441.19
3,1990-01-04,5444.15
4,1990-01-05,5447.11
...,...,...
12878,2025-04-05,38919.16
12879,2025-04-06,38924.18
12880,2025-04-07,38929.19
12881,2025-04-08,38934.20


In [5]:
def add_ordinal_date(df, date):
    df_output = df[df.date>=date].copy(deep=True)
    df_output['date'] = pd.to_datetime(df_output['date'])
    df_output['ordinal_date']=df_output['date'].map(dt.datetime.toordinal)
    return df_output

In [33]:
filtered_df = add_ordinal_date(df, '1990-01-01')

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    filtered_df[['ordinal_date']], filtered_df[['clfclp']], test_size=0.33, random_state=42)

reg = LinearRegression().fit(X_train.values, y_train)

# Make predictions using the testing set
y_pred = reg.predict(X_test.values)

# The coefficients
print("Coefficients: \n", reg.intercept_, reg.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [-1511378.58161952] [[2.08960517]]
Mean squared error: 2246489.16
Coefficient of determination: 0.96


In [49]:
dateee = dt.datetime.toordinal(dt.datetime(2025,3,18))
uf_navidad = reg.predict([[dateee]])

In [50]:
uf_navidad

array([[33525.03149665]])

In [48]:
# Crear fechas futuras para predicción (12 años desde el último dato)
ultima_fecha = filtered_df['date'].max()
fechas_futuras = pd.date_range(start=ultima_fecha, periods=365*30, freq='D')
fechas_ordinales_futuras = [dt.datetime.toordinal(date) for date in fechas_futuras]

# Hacer predicciones
predicciones_futuras = reg.predict([[fecha] for fecha in fechas_ordinales_futuras])

# Crear el gráfico
plt.figure(figsize=(300, 60))

# Datos de entrenamiento
plt.scatter(X_train, y_train, color='green', label='Datos de entrenamiento', alpha=1, s=1)
# Datos de prueba
plt.scatter(X_test, y_test, color='blue', label='Datos de prueba', alpha=1, s=1)

# Generar puntos para la recta de regresión
X_ordenado = np.concatenate([X_train.values, X_test.values])
X_ordenado.sort(axis=0)
y_ordenado = reg.predict(X_ordenado)

# Recta de regresión
plt.plot(X_ordenado, y_ordenado, color='black', label='Recta de regresión', linewidth=2)

# Predicciones futuras
plt.plot(fechas_ordinales_futuras, predicciones_futuras, 
         color='red', label='Predicción futura', linestyle='--')

plt.title('Valor de la UF: Datos históricos y Predicción')
plt.xlabel('Fecha')
plt.ylabel('Valor UF (CLP)')
plt.legend()
plt.grid(True, alpha=0.3)

# Convertir fechas ordinales a fechas reales en el eje x
ax = plt.gca()
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: dt.datetime.fromordinal(int(x)).strftime('%Y-%m-%d')))

# Rotar las etiquetas del eje x para mejor legibilidad
plt.xticks(rotation=45)

# Ajustar el diseño para que no se corten las etiquetas
plt.tight_layout()

plt.show()